<a href="https://colab.research.google.com/github/AlbinaAzMikhaylova/-Python/blob/main/%D0%91%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%20%22%D0%A1%D0%BA%D0%BB%D0%B5%D0%B8%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%20%D1%84%D0%B0%D0%B9%D0%BB%D0%BE%D0%B2%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1 кейс

**Ваша задача написать функцию `process_files`, которая принимает на вход два пути к папкам. Из первой папки необходимо выбрать все "чеки" (файлы по шаблону из условия), а во вторую папку сохранить один объединенный чек (отсортированный по дате, а затем по продукту) под названием `combined_data.csv`.**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить папку с файлами. После выполнения, в папке `reports_main` будут храниться все присланные магазинами чеки.

In [1]:
!wget https://github.com/vs8th/reports/archive/main.zip

import zipfile

with zipfile.ZipFile("main.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")

!rm main.zip

--2025-02-27 05:07:00--  https://github.com/vs8th/reports/archive/main.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Vs8th/reports/zip/refs/heads/main [following]
--2025-02-27 05:07:00--  https://codeload.github.com/Vs8th/reports/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.121.9
Connecting to codeload.github.com (codeload.github.com)|140.82.121.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [ <=>                ]   3.22K  --.-KB/s    in 0s      

2025-02-27 05:07:00 (68.3 MB/s) - ‘main.zip’ saved [3296]



Чтобы посмотреть как выглядят подходящие для объединения чеки выполните следующую ячейку.

In [2]:
import pandas as pd

df = pd.read_csv('reports-main/2023-02-17-05-38-2.csv', sep=";")
df

,date,product,store,cost
0,2023-02-17,product_0,store_2,10
1,2023-02-17,product_1,store_2,20
2,2023-02-17,product_2,store_2,30
3,2023-02-17,product_3,store_2,40
4,2023-02-17,product_4,store_2,50


**Решение**

Напишите свое решение ниже

**Примечание**

Не все файлы подходящие по названию, будут подходить по содержанию. Там может оказаться лишний столбец, например. Ориентируйтесь на столбцы из чека выше - это то, что вас интересует. Остальные столбцы можно просто отбросить.

**Важно**: разделителем файла на выходе должна быть запятая.

In [3]:
import glob
import os,sys
import re
import csv
import pandas as pd


In [9]:
def process_files(src_folder, dest_folder):
#  src_folder='reports-main'
  all_files = glob.glob(src_folder + '/*.csv')
  print(f"Найдено файлов: {len(all_files)}")  # Отладочный вывод
  # задаем шаблон имени для поиска
  re_name = r'reports-main/\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d+.csv'

  # Определяем нужные столбцы
  required_columns = ['date',	'product',	'store',	'cost']

  # перебираем все файлы CSV и выбираем те, чьё имя подходит по шаблону
  true_files=[]
  for file in all_files:
    if re.match(re_name, file):
      true_files.append(file)
  #print(f"Подходящие файлы: {true_files}")  # Отладочный вывод

  #Перебираем файлы
  new_csv=[]
  for file in true_files:
    with open(file, mode='r', newline='') as infile:
      my_file = infile.readline()
      #print(my_file)
      header = my_file.strip().split(';') # заголовок
      #print(header)
      if header == required_columns:
        for i in infile.readlines():
          new_csv.append(i.strip().split(';'))
          # отобрали файлы с подходящими столбцами

      else:
        for line in infile:
          data = line.strip().split(';')
          # формируем словарь заголовок:значение
          d = dict(zip(header, data))
          #print(d)
          row = []
          # выбираем построчно те ключи, которые попадают в список столбцов
          for k, v in d.items():
            if k in required_columns:
              row.append(v)
              # добавляем значения, оборачиваем их в список и добавляем в список
          new_csv.append(row)

  new_csv = sorted(new_csv, key=lambda x: x[0]) # отсортировали полученный список

#  создаём файл и папку
  pth = os.path.join(dest_folder, 'combined_data.csv')
  print(pth)

  if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

#  записываем строки в файл. сначала заголовки, потом содержимое
  with open(pth, mode='w', newline='') as outfile:
    outfile.write(','.join(required_columns)+'\n')
    for i in new_csv:
      outfile.write(','.join(i)+'\n')

src_folder = 'reports-main'
dest_folder = 'comb_reports'
process_files(src_folder, dest_folder)



Найдено файлов: 13
comb_reports/combined_data.csv


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [5]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv

import pandas as pd

user_answer = pd.read_csv(f'{dest_folder}/combined_data.csv')
correct_answer = pd.read_csv('data.csv')

--2025-02-27 05:07:13--  https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv
Resolving gist.github.com (gist.github.com)... 140.82.121.4
Connecting to gist.github.com (gist.github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv [following]
--2025-02-27 05:07:13--  https://gist.githubusercontent.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 984 [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]     984  --.-KB/s    in 0s      

2025-02-27 05:07:13 (53.0 MB/s) - ‘data.csv’ saved [984/984]



In [6]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!')

Поздравляем, Вы справились и успешно прошли все проверки!
